<a href="https://colab.research.google.com/github/AbbasHussain72/Wavelet-featre-extraction-/blob/main/CNN_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h5py
import pandas as pd
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.widgets import SpanSelector
import glob
import os
import re
import time

from numpy import mean
from numpy import std
import seaborn as sns
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Sequential
from keras.layers import Input, Activation, Dense, Dropout, Flatten,Conv1D,MaxPooling1D,concatenate
from keras.models import load_model,Model
from keras.utils import to_categorical , plot_model
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

import matplotlib as mpl
import numpy as np
import matplotlib.font_manager as fm
from matplotlib import font_manager as fm, pyplot as plt
!wget https://github.com/trishume/OpenTuringCompiler/blob/master/stdlib-sfml/fonts/Times%20New%20Roman.ttf
!wget https://github.com/matomo-org/travis-scripts/blob/master/fonts/Arial.ttf
font_files = fm.findSystemFonts()
# Go through and add each to Matplotlib's font cache.
for font_file in font_files:
    fm.fontManager.addfont(font_file)
# Use your new font on all your plots.
plt.rc('font', family='serif ')

In [ ]:
# !pip install tensorflow
# !pip install scikeras

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [131]:
#read datafile and reset index
path = '/content/drive/MyDrive/50L3_wfeature.csv'
df = pd.read_csv(path,sep='\t')
df = df.drop(['Unnamed: 0'], axis = 1)

#read wavelet names file
path = '/content/drive/MyDrive/wavelet_names.csv'
md = pd.read_csv(path,sep=',')
wavelets= md['Wavelets'].tolist()

In [ ]:
# df.head()

In [76]:
# print('X_train_reshaped: ', X_train_reshaped.shape)
# print('X_test_reshaped: ', X_test_reshaped.shape)

# print('y_train_encode : ', y_train_encode.shape)
# print('y_test: ', y_test.shape)

X_train_reshaped:  (1453, 55, 1)
X_test_reshaped:  (624, 55, 1)
y_train_encode :  (1453,)
y_test:  (624,)


In [77]:
# #To check the unique value
# unique_values_train = np.unique(y_train_encode)
# unique_values_test = np.unique(y_test)
# print("Unique values in y_train:", unique_values_train)
# print("Unique values in y_test:", unique_values_test)


Unique values in y_train: [0 1 2 3 4 5 6 7 8 9]
Unique values in y_test: ['OP01bad' 'OP01good' 'OP02bad' 'OP02good' 'OP04bad' 'OP04good' 'OP07bad'
 'OP07good' 'OP10bad' 'OP10good']


# CNN model

In [54]:
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(55, 1)))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Assuming 10 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32)


In [56]:
y_pred = model.predict(X_test_reshaped)
y_pred_labels = np.argmax(y_pred, axis=1)

20/20 [==============================] - 0s 2ms/step


In [57]:
# Invert the class labels using the inverse_transform method
y_pred_original_labels = label_encoder.inverse_transform(y_pred_labels)

In [58]:
np.unique(y_pred_original_labels)

array(['OP01bad', 'OP01good', 'OP02bad', 'OP02good', 'OP04bad',
       'OP04good', 'OP07bad', 'OP07good', 'OP10bad', 'OP10good'],
      dtype=object)

# Hyperparameter Tuning Using gridsearch

In [ ]:
wavelets

In [ ]:
wavelets1 = wavelets[0:15]
wavelets1

In [ ]:
# Iterate data through each wavelet

# Create an empty DataFrame to store results
columns = ['wavelet', 'drop_rate', 'kernel_size', 'learning_rate', 'num_filters', 'val_f1_score']
df_results = pd.DataFrame(columns=columns)

f1_scores_list = []

for w in wavelets1:
  print(f"\nProcessing for wavelet: {w}")
  start_time = time.time()
  new = df[df['wavelet'] == w]
  # print (new.shape)
  X = new.drop(['Date', 'Machine', 'Operation', 'Condition', 'Repeat', 'operationcondition', 'wavelet'], axis=1)
  y = new['operationcondition']
  print (w)
  #Train_Test split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  # # Create for feature selection
  f_selection = RandomForestClassifier(random_state = 42, verbose=0)
  # Perform feature selection using SelectFromModel
  feature_selector = SelectFromModel(f_selection)
  feature_selector.fit(X_train, y_train)
  X_train_transformed = feature_selector.transform(X_train)
  X_test_transformed = feature_selector.transform(X_test)

  # Perform standardscalar transformation
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train_transformed)
  X_test_scaled = scaler.transform(X_test_transformed)

  #Label Encoding for CNN
  label_encoder = LabelEncoder()
  y_train_encode = label_encoder.fit_transform(y_train)

  # Reshape the data for a 1D CNN
  X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
  X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

  inputshape = (X_train_scaled.shape[1],1)

  def create_model(num_filters=64, kernel_size=3, dropout_rate=0.5, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(num_filters, kernel_size, activation='relu', input_shape= inputshape ))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))  # Add dropout here with the specified rate
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

  # Wrap the Keras model for use with scikit-learn
  keras_model = KerasClassifier(build_fn=create_model,num_filters=16,learning_rate=0.001, dropout_rate=0.2,kernel_size=3, epochs=10, batch_size=32, verbose=0)

  # Define the parameter grid for grid search
  param_grid = {
    'num_filters': [16, 32, 64],
    'kernel_size': [3, 5,7],
    'dropout_rate': [0.2, 0.5, 0.8],
    'learning_rate': [0.001, 0.01,0.1]}

  # Create GridSearchCV instance
  grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, verbose = 0,
                          scoring='f1_macro', cv=3, n_jobs=-1)


  # Fit the grid search to your data
  grid_search.fit(X_train_reshaped, y_train_encode)

  # print(f'Best hyperparameters for {w}: {grid_search.best_params_}')
  # print(f'Best hyperparameters for {w}: {grid_search.best_score_}')

  # Extract best parameters and score
  best_params = grid_search.best_params_
  best_score = grid_search.best_score_

  # Add a row with values to the DataFrame
  df_results = df_results.append({
        'wavelet': w,
        'drop_rate': best_params['dropout_rate'],
        'kernel_size': best_params['kernel_size'],
        'learning_rate': best_params['learning_rate'],
        'num_filters': best_params['num_filters'],
        'f1_score': best_score
    }, ignore_index=True)

  # Save the DataFrame to a CSV file after each iteration
  df_results.to_csv('/content/drive/MyDrive/CNNl3_bestparascore.csv', sep='\t', index=False)


  # Prediction on the test set
  y_pred = grid_search.predict(X_test_reshaped)

  # # Reshape y_pred if needed
  # if y_pred.ndim == 1:
  #     y_pred = y_pred.reshape(-1, 1)

  # y_pred_labels = np.argmax(y_pred, axis=1)
  y_pred_original_labels = label_encoder.inverse_transform(y_pred)
  end_time = time.time()

  #Evaluation on f1score
  f1_test = f1_score(y_test,y_pred_original_labels,average='macro')
  print(end_time-start_time)
  # print(f'F1 score on test set for {w}: {f1_test}')

  # Save the F1 score for the current wavelet to the list
  f1_scores_list.append({'wavelet': w, 'f1_score': f1_test})

# Save the final data frame to a CSV file
df_results.to_csv('/content/drive/MyDrive/CNNl3_bestparascore_final.csv', sep='\t', index=False)

# Create the DataFrame outside the loop
df_f1_scores = pd.DataFrame(f1_scores_list)
# Save the DataFrame to a CSV file
df_f1_scores.to_csv('/content/drive/MyDrive/CNNl3f1scores_final.csv', sep='\t', index=False)






Processing for wavelet: db2
db2


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
<ipython-input-133-b0cea9a867d6>:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


363.4583203792572

Processing for wavelet: db3
db3


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
<ipython-input-133-b0cea9a867d6>:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


353.65211629867554

Processing for wavelet: db4
db4


...

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
<ipython-input-133-b0cea9a867d6>:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


346.9253628253937

Processing for wavelet: db6
db6


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
<ipython-input-133-b0cea9a867d6>:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append({


345.86429262161255

Processing for wavelet: db7
db7


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [124]:
dfread = pd.read_csv('/content/drive/MyDrive/CNNl3f1scores.csv',sep='\t')

In [125]:
dfread

,wavelet,f1_score
0,db10,0.9129


# Confusion Matrix for CNN

In [60]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_test_pred = y_pred_original_labels
test_confu_matrix = confusion_matrix(y_test,  y_test_pred)
report = classification_report(y_test,  y_test_pred)
# print(report)
lines = report.split('\n')   # Split the report into lines
labels = [line.split()[0] for line in lines[2:12]]   # Extract the labels from each line

In [61]:
print(report)

              precision    recall  f1-score   support

     OP01bad       0.95      0.97      0.96        64
    OP01good       0.85      0.94      0.89        53
     OP02bad       1.00      0.89      0.94        65
    OP02good       0.85      0.98      0.91        64
     OP04bad       0.91      0.88      0.90        60
    OP04good       0.82      0.84      0.83        49
     OP07bad       0.90      0.96      0.93        76
    OP07good       0.93      0.86      0.90        65
     OP10bad       0.95      0.88      0.91        60
    OP10good       0.95      0.88      0.92        68

    accuracy                           0.91       624
   macro avg       0.91      0.91      0.91       624
weighted avg       0.92      0.91      0.91       624



In [62]:
dd = pd.DataFrame([row.split() for row in report.split('\n')[2:-5]])

In [ ]:
dd.head(10)